In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()
tf.VERSION
import sklearn
print(sklearn.__version__)


In [ ]:
import pathlib

DATADIR = 'C:/Michael/Labeled dataset'
data_root = pathlib.Path(DATADIR)

In [ ]:

all_image_paths = list(data_root.glob('*/*.jpg'))
all_image_paths = [str(path) for path in all_image_paths]

all_xml_paths = list(data_root.glob('*/*.xml'))
all_xml_paths = [str(path) for path in all_xml_paths]

image_count = len(all_image_paths)
xml_count = len(all_xml_paths)
image_count# + xml_count

In [ ]:
all_image_paths[:100] + all_xml_paths[:10]

In [ ]:
import IPython.display as display

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return image_rel

In [ ]:
import random

for n in range(10):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(caption_image(image_path))

In [ ]:
#preprocess training images

import xml.etree.ElementTree as ET
import numpy as np
from array import *
from PIL import Image
import uuid

def find_coeffs(source_coords, target_coords):
    matrix = []
    for s, t in zip(source_coords, target_coords):
        matrix.append([t[0], t[1], 1, 0, 0, 0, -s[0]*t[0], -s[0]*t[1]])
        matrix.append([0, 0, 0, t[0], t[1], 1, -s[1]*t[0], -s[1]*t[1]])
    A = np.matrix(matrix, dtype=np.float)
    B = np.array(source_coords).reshape(8)
    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)

def xml_tree(x):
    for child in x:
        if(child.getchildren() == []):
            print(child.tag, ":", child.text)
        else:
            print(child.tag)
            xml_tree(child)

size = 640, 640

for i in range(len(all_image_paths)):
    tree = ET.parse(all_xml_paths[i])
    root = tree.getroot()
    xml_tree(root)
    print("")

    objs = root.findall("./object")

    x_list = []
    y_list = []
    for co in objs:
        print(co.find("./name").text)
        for node in co.findall("./polygon/pt"):
            print("point")
            children = node.getchildren()
            x = children[0].text
            y = children[1].text
            x_list.append(int(float(x)))
            y_list.append(int(float(y)))
            print(int(float(x)), int(float(y)))
        print("top-left", min(x_list), min(y_list))
        print("bottom-right", max(x_list), max(y_list))
        print()
    
        im = Image.open(all_image_paths[i])
        if(str(co.find("./name").text) == 'no pothole'):
            outfile = 'train/pothole0-%s.png' % (str(uuid.uuid4()))
        elif(str(co.find("./name").text) == 'pothole'):
            if(co.find("./attributes").text == None):
                outfile = 'train/pothole3-%s.png' % (str(uuid.uuid4()))
            else:
                outfile = 'train/pothole%s-%s.png' % (str(co.find("./attributes").text), str(uuid.uuid4()))
        else:
            if(str(co.find("./name").text) == 'pothole1' or str(co.find("./name").text) == 'pothole2' 
               or str(co.find("./name").text) == 'pothole3' or str(co.find("./name").text) == 'pothole4'):
                outfile = 'train/%s-%s.png' % (str(co.find("./name").text), str(uuid.uuid4()))
            elif(str(co.find("./name").text) == 'crack'):
                outfile = 'train/pothole1-%s.png' % (str(uuid.uuid4()))
            else:
                outfile = 'train/pothole0-%s.png' % (str(uuid.uuid4()))
       # im.crop((min(x_list), min(y_list), max(x_list), max(y_list))).resize((640, 640)).save(outfile)
        coeffs = find_coeffs([(0, 0), (im.width, 0), (im.width, im.height), (0, im.height)], [(0, 0), (640, 0), (640, 640), (0, 640)])

        im = im.transform(size, Image.PERSPECTIVE, coeffs, Image.BICUBIC)
        
        im.save(outfile)
        print("Save as:", outfile)
        print()
        
        x_list = []
        y_list = []

In [ ]:
#display some training images
import pathlib
import IPython.display as display
from PIL import Image
import os
import random


data_root = pathlib.Path('C:/Users/mike')
training_objs = list(data_root.glob('test/*.png'))
training_objs = [str(path) for path in training_objs]
random.shuffle(training_objs)

for i in range(10):
    display.display(display.Image(training_objs[i]))
    print(os.path.basename(training_objs[i])[0:8])

In [ ]:
#prepare training arrays

from __future__ import absolute_import, division, print_function

from sklearn import svm
from sklearn import tree
import tensorflow as tf
import pathlib
import random
import numpy as np
import os

tf.enable_eager_execution()

#clf = svm.SVC(C=1,probability=True,gamma=.1) 
#clf = tree.DecisionTreeClassifier() 
clf = svm.SVC(kernel='linear',probability=True) 

data_root = pathlib.Path('C:/Users/mike')
training_objs = list(data_root.glob('test/*.png'))
training_objs = [str(path) for path in training_objs]
random.shuffle(training_objs)

X = []
Y = []

for i in range(len(training_objs)):
    img_raw = tf.read_file(training_objs[i])
    img_tensor = tf.image.decode_image(img_raw, channels=3)
    img_tensor = tf.image.rgb_to_grayscale(img_tensor)
    img_tensor = tf.reshape(img_tensor, [409600])

    a = np.array(img_tensor).tolist()
    X.append(a)
    Y.append(os.path.basename(training_objs[i])[0:8])


In [ ]:
#optimizing hyperparameters
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y )
    grid_search.best_params_
    return grid_search.best_params_
svc_param_selection(X,Y, 3)

In [ ]:
#number of training images
len(Y)

In [ ]:
#train the model

clf = clf.fit(X, Y)


In [ ]:
import pathlib
import IPython.display as display
import random

DATADIR = 'C:/Michael/ExtraData1'
data_root = pathlib.Path(DATADIR)

all_image_paths = list(data_root.glob('*/*.jpg'))
all_image_paths = [str(path) for path in all_image_paths]
all_xml_paths = list(data_root.glob('*/*.xml'))
all_xml_paths = [str(path) for path in all_xml_paths]

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return image_rel
    
# for n in range(30):
#   image_path = random.choice(all_image_paths)
#   display.display(display.Image(image_path))
#   print(caption_image(image_path))

In [ ]:
#preprocess labeled testing images

import xml.etree.ElementTree as ET
import numpy as np
from array import *
from PIL import Image
import uuid
def find_coeffs(source_coords, target_coords):
    matrix = []
    for s, t in zip(source_coords, target_coords):
        matrix.append([t[0], t[1], 1, 0, 0, 0, -s[0]*t[0], -s[0]*t[1]])
        matrix.append([0, 0, 0, t[0], t[1], 1, -s[1]*t[0], -s[1]*t[1]])
    A = np.matrix(matrix, dtype=np.float)
    B = np.array(source_coords).reshape(8)
    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)
def xml_tree(x):
    for child in x:
        if(child.getchildren() == []):
            print(child.tag, ":", child.text)
        else:
            print(child.tag)
            xml_tree(child)
size = 640,640
for i in range(len(all_image_paths)):
    tree = ET.parse(all_xml_paths[i])
    root = tree.getroot()
    xml_tree(root)
    print("")

    objs = root.findall("./object")

    url = 'C:/Michael/ExtraData1/Image/' + root.find("./filename").text

    x_list = []
    y_list = []
    
    for co in objs:
        print(co.find("./name").text)
        for node in co.findall("./polygon/pt"):
            print("point")
            children = node.getchildren()
            x = children[0].text
            y = children[1].text
            x_list.append(int(float(x)))
            y_list.append(int(float(y)))
            print(int(float(x)), int(float(y)))
        print("top-left", min(x_list), min(y_list))
        print("bottom-right", max(x_list), max(y_list))
        print()
    
        im = Image.open(url) 
        outfile = 'testing_objects/%s-%s.png' % (str(co.find("./name").text), str(uuid.uuid4()))
        coeffs = find_coeffs([(0, 0), (im.width, 0), (im.width, im.height), (0, im.height)], [(0, 0), (640, 0), (640, 640), (0, 640)])

        im = im.transform(size, Image.PERSPECTIVE, coeffs, Image.BICUBIC)
        im.save(outfile)
        print("Save as:", outfile)
        print()
        
        x_list = []
        y_list = []

In [ ]:
#preprocess unlabeled testing images

import numpy as np
from array import *
from PIL import Image
import uuid

def xml_tree(x):
    for child in x:
        if(child.getchildren() == []):
            print(child.tag, ":", child.text)
        else:
            print(child.tag)
            xml_tree(child)
            
for n in range(43):
    image_path = random.choice(all_image_paths)
    display.display(display.Image(image_path))
    print(caption_image(image_path))
    
    im = Image.open(image_path) 
    outfile = 'testing_objects/%d.png' % (n)
    coeffs = find_coeffs([(0, 0), (im.width, 0), (im.width, im.height), (0, im.height)], [(0, 0), (640, 0), (640, 640), (0, 640)])

    im = im.transform(size, Image.PERSPECTIVE, coeffs, Image.BICUBIC)
    im.save(outfile)
    print("Save as:", outfile)
    print()


In [ ]:
#prepare test array

from __future__ import absolute_import, division, print_function

from sklearn import svm
import tensorflow as tf
import pathlib
import random
import numpy as np
import os

tf.enable_eager_execution()

data_root = pathlib.Path('C:/Users/mike')
testing_objs = list(data_root.glob('testing_objects2/*.png'))
testing_objs = [str(path) for path in testing_objs]
random.shuffle(testing_objs)

A = []
B = []

for i in range(20):
    img_raw = tf.read_file(testing_objs[i])
    img_tensor = tf.image.decode_image(img_raw, channels=3)
    img_tensor = tf.image.rgb_to_grayscale(img_tensor)
    img_tensor = tf.reshape(img_tensor, [409600])

    a = np.array(img_tensor).tolist()
    A.append(a)
    B.append(os.path.basename(testing_objs[i])[0:8])

In [ ]:
#number of test images

len(testing_objs)

In [ ]:
for i in range(13):
    display.display(display.Image(testing_objs[i]))
    print(os.path.basename(testing_objs[i])[0:8])

In [ ]:
#predict test images

predictions = []
print(clf.predict_proba(A))
predictions = clf.predict(A)

In [ ]:

for i in range(13):
    print(predictions[i])

In [ ]:
print(clf.score(A,B))


In [ ]:

import IPython.display as display
import pathlib
from PIL import Image
import os
import random

#for n in range(10):
data_root = pathlib.Path('C:/Users/mike')
testing_objs = list(data_root.glob('testing_objects2/*.png'))
testing_objs = [str(path) for path in testing_objs]
#random.shuffle(testing_objs)

# for i in range(13):
#     display.display(display.Image(testing_objs[i]))
#     print(os.path.basename(testing_objs[i])[0:8])
prediction = []
accuracy = 0
accuracy1 = 0
for i in range(100):
    img_raw = tf.read_file(testing_objs[i])
    img_tensor = tf.image.decode_image(img_raw)
    img_tensor = tf.image.rgb_to_grayscale(img_tensor)
    img_tensor = tf.reshape(img_tensor, [-1, 409600])

    a = np.array(img_tensor.numpy()).tolist() 

    prediction = clf.predict(a)
    display.display(display.Image(testing_objs[i]))
    print(os.path.basename(testing_objs[i])[0:8])
    print(prediction)
    if(os.path.basename(testing_objs[i])[0:8] == prediction):
        accuracy += 1
        accuracy1 += 1


#     if(os.path.basename(testing_objs[i])[0:8] == prediction and os.path.basename(testing_objs[i])[0:8] == 'pothole0'):
#         accuracy1 += 1
    elif(os.path.basename(testing_objs[i])[0:8] != 'pothole0' and prediction != 'pothole0'):
        accuracy1 += 1

    print('accuracy: ' + str(accuracy))
    print('accuracy1: ' + str(accuracy1))
    print(i)
print(accuracy/100)
print(accuracy1/100)

In [ ]:
print('level accuracy: ' + str(accuracy) +'%')
print('pothole accuracy: '+ str (accuracy1) +'%')